In [ ]:
import mne
from sklearn.decomposition import PCA
import numpy as np

#file_path = 'DATA/Data_Inner_Layer/PTW07_allwalk_EEG.set'
file_path = 'DATA/Data_Artifact_Layer/PTW01_allwalk_artifact.set'
raw = mne.io.read_raw_eeglab(file_path, preload=True)

In [ ]:
import mne
import numpy as np
from joblib import Parallel, delayed


# Extract data for channel 'A1'
data_a1 = raw.copy().pick_channels(['A1']).get_data()[0]

channels = raw.ch_names

def compute_correlation(channel_data):
    """Computes the correlation between A1 and the provided channel data."""
    return np.corrcoef(data_a1, channel_data)[0, 1]

# Use joblib to compute correlations in parallel
correlations = Parallel(n_jobs=-1)(delayed(compute_correlation)(raw.copy().pick_channels([ch]).get_data()[0]) for ch in channels)

for ch, corr in zip(channels, correlations):
    print(f"Correlation between A1 and {ch}: {corr:.2f}")


In [ ]:
erp =all_data[0].average()
# Plot the ERP
erp.plot(); 

In [ ]:
import mne
import numpy as np
from joblib import Parallel, delayed


epochs = all_data[0]

channels = all_data[0].ch_names

def compute_correlation_for_epoch(epoch, channel):
    """Computes the correlation between A1 and a given channel for a single epoch."""
    data_a1 = epoch.copy().pick_channels(['A1']).get_data().squeeze()
    data_ch = epoch.copy().pick_channels([channel]).get_data().squeeze()
    return np.corrcoef(data_a1, data_ch)[0, 1]

correlations_for_all_epochs = []

# Compute correlations for each channel across all epochs
for ch in channels:
    correlations = Parallel(n_jobs=-1)(delayed(compute_correlation_for_epoch)(epochs[i], ch) for i in range(len(epochs)))
    correlations_for_all_epochs.append(np.mean(correlations))  # Average correlations across all epochs

for ch, corr in zip(channels, correlations_for_all_epochs):
    print(f"Average correlation between A1 and {ch}: {corr:.2f}")


In [ ]:
events = mne.events_from_annotations(raw)